<!-- ---
title: Hyperparameter Tuning in Ignite
date: 2021-09-28
downloads: true
sidebar: true
tags:
  - hyperparameter tuning
  - ray tune
  - optuna
  - ax
--- -->

#  Hyperparameter Tuning in Ignite

In this tutorial, we will see how [Ray Tune](https://docs.ray.io/en/stable/tune.html) can be used with Ignite for hyperparameter tuning. We will also compare it with other frameworks like [Optuna](https://optuna.org/) and [Ax](https://ax.dev/) for hyperparameter optimization.

<!--more-->

We will follow [this PyTorch tutorial](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html) for reference.

## Required Dependencies

In [1]:
!pip install pytorch-ignite ray[tune] -q

## Imports

In [21]:
import os
from functools import partial
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split

from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18
from torchvision.transforms import Compose, Normalize, ToTensor

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage
import ignite.distributed as idist

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

## Dataloaders

First, we'll load our the CIFAR10 dataset from a global `data_dir` and then split the train dataset into `train_subset` and `val_subset`. We use `auto_dataloader` to wrap these datasets so they can easily adapt to a distributed configuraion. Note that we are making the `batch_size` configurable so that we can tune them later.

In [22]:
def load_data(data_dir="./data"):
    transform = Compose([
        ToTensor(),
        Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [35]:
def get_train_val_loaders(config, data_dir=None):
    trainset, _ = load_data(data_dir)
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = idist.auto_dataloader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = idist.auto_dataloader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    
    return trainloader, valloader

## Model, Optimizer and Criterion

We will also make the last two fully connected layers of our model configurable.

In [36]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

`config` receives the hyperparameters we would use in the current trial. Now we wrap the model and optimizer in the `auto_*` methods to make them easily adaptable when they are running on a distributed configuration. We also make sure that if a checkpoint directory is provided, we load the state of the model and optimizer first, then continue.

In [42]:
def initialize(config, checkpoint_dir):
    model = idist.auto_model(Net(config["l1"], config["l2"]))

    device = idist.device()

    criterion = nn.CrossEntropyLoss()
    optimizer = idist.auto_optim(optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9))

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    
    return model, device, criterion, optimizer

## Train One Trial

Now let's create the training API by wrapping the steps needed to train the above model in a function. 

1. Get the dataloaders, model, device, criterion and optimizer to initialize the `trainer` and `val_evaluator`.
2. We attach an additional metric `RunningAverage()` to the `trainer` so that we can log the loss every 2000 iterations.
3. The `val_evaluator` also records the accuracy and loss metrics after an epoch is completed.
4. We can additionally save the model and optimizer's state if we want to test them later or use advanced schedulers.

Now here is how Ray Tune uses the metrics from our evaluator to tune the hyperparameters:
```
tune.report(loss=metrics['loss'], accuracy=metrics['accuracy'])  
```

The loss and accuracy metrics gets reported and Ray Tune can decide which hyperparameter configuration leads to the the best trial. It can also prematurely terminate a trial if it doesn't see the metrics improving. This is a huge performance gain when training on large networks or data.

5. Finally, we run the `trainer` for a set number of epochs.

In [43]:
def train_cifar(config, data_dir=None, checkpoint_dir=None):
    trainloader, valloader = get_train_val_loaders(config, data_dir)
    model, device, criterion, optimizer = initialize(config, checkpoint_dir)
    
    trainer = create_supervised_trainer(model, optimizer, criterion, device=device, non_blocking=True)
    
    avg_output = RunningAverage(output_transform=lambda x: x)
    avg_output.attach(trainer, 'running_avg_loss')
    
    val_evaluator = create_supervised_evaluator(model, metrics={ "accuracy": Accuracy(), "loss": Loss(criterion)}, device=device, non_blocking=True)
    
    @trainer.on(Events.ITERATION_COMPLETED(every=2000))
    def log_training_loss(engine):
        print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f} Running Avg Loss: {engine.state.metrics['running_avg_loss']:.2f}")


    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        val_evaluator.run(valloader)
        metrics = val_evaluator.state.metrics
        print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

        with tune.checkpoint_dir(trainer.state.epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)
            
        tune.report(loss=metrics['loss'], accuracy=metrics['accuracy'])   

    trainer.run(trainloader, max_epochs=10) 

## Test Results

It is also a good practice to test the results after all trials are over to get the final metrics.

In [44]:
def test_best_model(best_trial, data_dir=None):
    _, testset = load_data(data_dir)
    
    best_trained_model = idist.auto_model(Net(best_trial.config["l1"], best_trial.config["l2"]))
    device = idist.device()

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_evaluator = create_supervised_evaluator(best_trained_model, metrics={"Accuracy": Accuracy()}, device=device, non_blocking=True)

    testloader = idist.auto_dataloader(testset, batch_size=4, shuffle=False, num_workers=2)

    test_evaluator.run(testloader)
    print(f"Best trial test set accuracy: {test_evaluator.state.metrics}")

## Train for All Trials

Before we begin training, we need to define Ray Tune's search. As we can see below the configurable parameters are:
* `l1` and `l2`: The last fully connected layers of the model whose values should be powers of 2, anywhere between 4 and 256. For example - 4, 8, 16, etc.
* `lr`: The learning rate which should be uniformly sampled between 0.0001 and 0.1.
* `batch_size`: The batch size should be a choice between 2, 4, 8 and 16.

For every trial, Ray Tune will randomly sample a combination of parameters from these search spaces. It will then train a number of models in parallel and find the best performing one among these. 
We also use the `ASHAScheduler()` which is one of the trial schedulers that aggressively terminate low-performing trials.
Apart from that, we leverage the `CLIReporter()` to prettify our outputs.
And then, we wrap `train_cifar` in functools.partial and pass it to `tune.run` along with other resources like the CPUs and GPUs available to use, the configurable parameters, the number of trials, scheduler and reporter.
We can extract the best trial according to the metric (loss below), mode (minimum below) and scope (last trial below).

And at last, we print the metrics of the best model.

In [45]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    
    config = {
        "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")
    
    test_best_model(best_trial, data_dir)

Finally, we begin training.

In [46]:
main(num_samples=10, max_num_epochs=10, gpus_per_trial=2)

Files already downloaded and verified
Files already downloaded and verified


2021-11-08 14:00:56,824	WARNING experiment.py:303 -- No name detected on trainable. Using DEFAULT.
2021-11-08 14:00:56,824	INFO registry.py:67 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 11.0/62.8 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/12 CPUs, 0/2 GPUs, 0.0/43.89 GiB heap, 0.0/9.31 GiB objects
Result logdir: /root/ray_results/DEFAULT_2021-11-08_14-00-56
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+--------------+------+------+-------------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-------+--------------+------+------+-------------|
| DEFAULT_e9ca1_00000 | PENDING  |       |            2 |    8 |   32 | 0.00051665  |
| DEFAULT_e9ca1_00001 | PENDING  |       |            8 |   16 |   64 | 0.00118436  |
| DEFAULT_e9ca1_00002 | PENDING  |       |            8 |  256 |    4 | 0.00202913  |
| DEFAULT_e9ca1_00003 | PENDING  |       |            2 |  128 |   32 | 0.00869588  |
| DEFAULT_e9ca1_00004 | PENDING  |       |  

== Status ==
Memory usage on this node: 14.0/62.8 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/12 CPUs, 2.0/2 GPUs, 0.0/43.89 GiB heap, 0.0/9.31 GiB objects
Result logdir: /root/ray_results/DEFAULT_2021-11-08_14-00-56
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+--------------+------+------+-------------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-------+--------------+------+------+-------------|
| DEFAULT_e9ca1_00000 | RUNNING  |       |            2 |    8 |   32 | 0.00051665  |
| DEFAULT_e9ca1_00001 | PENDING  |       |            8 |   16 |   64 | 0.00118436  |
| DEFAULT_e9ca1_00002 | PENDING  |       |            8 |  256 |    4 | 0.00202913  |
| DEFAULT_e9ca1_00003 | PENDING  |       |            2 |  128 |   32 | 0.00869588  |
| DEFAULT_e9ca1_00004 | PENDIN

== Status ==
Memory usage on this node: 14.0/62.8 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/12 CPUs, 2.0/2 GPUs, 0.0/43.89 GiB heap, 0.0/9.31 GiB objects
Result logdir: /root/ray_results/DEFAULT_2021-11-08_14-00-56
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+--------------+------+------+-------------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-------+--------------+------+------+-------------|
| DEFAULT_e9ca1_00000 | RUNNING  |       |            2 |    8 |   32 | 0.00051665  |
| DEFAULT_e9ca1_00001 | PENDING  |       |            8 |   16 |   64 | 0.00118436  |
| DEFAULT_e9ca1_00002 | PENDING  |       |            8 |  256 |    4 | 0.00202913  |
| DEFAULT_e9ca1_00003 | PENDING  |       |            2 |  128 |   32 | 0.00869588  |
| DEFAULT_e9ca1_00004 | PENDIN

...

== Status ==
Memory usage on this node: 20.4/62.8 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: -1.25606435546875 | Iter 2.000: -1.3796359375 | Iter 1.000: -1.51973603515625
Resources requested: 2.0/12 CPUs, 2.0/2 GPUs, 0.0/43.89 GiB heap, 0.0/9.31 GiB objects
Result logdir: /root/ray_results/DEFAULT_2021-11-08_14-00-56
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+------------------+--------------+------+------+-------------+--------+------------+----------------------+
| Trial name          | status   | loc              |   batch_size |   l1 |   l2 |          lr |   loss |   accuracy |   training_iteration |
|---------------------+----------+------------------+--------------+------+------+-------------+--------+------------+----------------------|
| DEFAULT_e9ca1_00000 | RUNNING  | 172.17.0.2:21411 |            2 |    8 |   32 | 0.00051665  | 1.2516 |     0.5537 |                    6 |
| DEFAULT_e9ca1_00001 | PENDI

2021-11-08 14:32:00,869	WARNING worker.py:1227 -- 3 retries left for task c0e56a21dcb5fe64ffffffffffffffffffffffff01000000, attempting to resubmit.
2021-11-08 14:32:00,909	WARNING worker.py:1227 -- 3 retries left for task 36e14f5d7a18682effffffffffffffffffffffff01000000, attempting to resubmit.
2021-11-08 14:32:00,994	ERROR tune.py:613 -- Trials did not complete: [DEFAULT_e9ca1_00000, DEFAULT_e9ca1_00001, DEFAULT_e9ca1_00002, DEFAULT_e9ca1_00003, DEFAULT_e9ca1_00004, DEFAULT_e9ca1_00005, DEFAULT_e9ca1_00006, DEFAULT_e9ca1_00007, DEFAULT_e9ca1_00008, DEFAULT_e9ca1_00009]
2021-11-08 14:32:01,001	INFO tune.py:617 -- Total run time: 1864.18 seconds (1863.78 seconds for the tuning loop).
2021-11-08 14:32:01,005	WARNING tune.py:622 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


Best trial config: {'l1': 8, 'l2': 32, 'lr': 0.0005166496334935304, 'batch_size': 2}
Best trial final validation loss: 1.2515984375
Best trial final validation accuracy: 0.5537
Files already downloaded and verified
Files already downloaded and verified


2021-11-08 14:32:05,923 ignite.distributed.auto.auto_model INFO: Apply torch DataParallel on model
2021-11-08 14:32:05,940 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 4, 'shuffle': False, 'num_workers': 2, 'pin_memory': True}
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Engine run is terminating due to exception: 


KeyboardInterrupt: 

And we can easily assess the results by loading Tensorboard.

In [47]:
%load_ext tensorboard
%tensorboard --logdir ~/ray_results/